1. mon_standard.pkl > array code



In [8]:
import pickle

USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

# Load the pickle file

with open("mon_standard.pkl", 'rb') as fi: 
    data = pickle.load(fi)

X1_mon = [] #타임스탬프 데이터를 저장할 배열
X2_mon = [] #방향 * 크기 데이터를 저장할 배열
y_mon = []  #레이블을 저장할 배열 (각 샘플이 어느 웹사이트에 속하는지가 표기될 예정)

# 웹사이트와 인스턴스를 구분하여 저장하는 루프
# x array (direction*timestamp), y array (site label)
for i in range(TOTAL_URLS):
    #레이블 할당 : 같은 웹사이트의 모든 하위 페이지에 동일한 레이블 부여
    if USE_SUBLABEL:
        label = i   #각 페이지에 고유한 레이블 부여
    else:
        label = i // URL_PER_SITE # Calculate which site's URL the current URL being processed belongs to and set that value as the label. Thus, URLs fetched from the same site are labeled identically.


    #각 페이지의 샘플(트래픽 데이터 처리하기)
    for sample in data[i]:   #해당 페이지의 모든 트래픽 샘플 반복
        size_seq = []        #크기 데이터 저장 배열
        time_seq = []        #타임스탬프 저장 배열

        #각 트래픽 샘플의 패킷 처리
        for c in sample:
            dr = 1 if c > 0 else -1     #방향 결정 (양수 : 1, 음수 : -1)
            time_seq.append(abs(c))     #절대값으로 타임스탬프 저장
            size_seq.append(dr * 512)   #방향에 512를 곱해 크기 저장

        #각 샘플의 데이터를 배열에 추가
        X1_mon.append(time_seq)
        X2_mon.append(size_seq)
        y_mon.append(label)

#전체 샘플 수 출력
size1 = len(X1_mon)
size2 = len(X2_mon)
size3 = len(y_mon)
print(f'Total samples: {size1}') # Output: 19000
print(f'Total samples: {size2}') # Output: 19000
print(f'Total samples: {size3}') # Output: 19000

Total samples: 19000
Total samples: 19000
Total samples: 19000


2. unmon_standard10.pkl > array code

In [9]:

import pickle

TOTAL_URLS = 3000  # total number in the dataset


with open('unmon_standard10_3000.pkl', 'rb') as f:  # Path to unmon_standard10.pkl in Colab
    x = pickle.load(f)

size = len(x)
print(f'Total samples: {size}')

X1_unmon = [] # Array to store instances (timestamps) - 10,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
X2_unmon = [] # Array to store instances (direction*size) - size information
y_unmon=[]

for i in range(TOTAL_URLS):
    size_seq = []
    time_seq = []

    for c in x[i]:
        dr = 1 if c > 0 else -1
        time_seq.append(abs(c))
        size_seq.append(dr * 512) # In the pickle file, there is no size information, so the conversion code is set to multiply by 512 uniformly.

    X1_unmon.append(time_seq)
    X2_unmon.append(size_seq)
    y_unmon.append(-1)

print(len(X1_unmon)) # Print the length of X1



Total samples: 3000
3000


In [10]:
import numpy as np

# 피처 추출 함수 정의
def extract_features(X1, X2):
    features = []

    for time_seq, size_seq in zip(X1, X2):
        total_packets = len(time_seq)   
        incoming_packets = sum(1 for s in size_seq if s > 0)
        outgoing_packets = total_packets - incoming_packets
        incoming_ratio = incoming_packets / total_packets
        outgoing_ratio = outgoing_packets / total_packets

        outgoing_order = [i for i, s in enumerate(size_seq) if s > 0]
        outgoing_std = np.std(outgoing_order)
        outgoing_mean = np.mean(outgoing_order)

        packets_per_second = total_packets / (time_seq[-1] - time_seq[0] + 1e-9)

        first_30_incoming = sum(1 for s in size_seq[:30] if s > 0)
        first_30_outgoing = 30 - first_30_incoming

        inter_arrival_times = np.diff(time_seq)
        inter_arrival_mean = np.mean(inter_arrival_times)
        inter_arrival_std = np.std(inter_arrival_times)

        chunk_size = 20
        concentration = [sum(1 for s in size_seq[i:i + chunk_size] if s > 0) 
                         for i in range(0, len(size_seq), chunk_size)]
        concentration_mean = np.mean(concentration)
        concentration_std = np.std(concentration)

        alternative_concentration = [sum(size_seq[i:i + chunk_size]) 
                                     for i in range(0, len(size_seq), chunk_size)]
        alternative_sum = sum(alternative_concentration)

        feature_vector = [
            total_packets, incoming_packets, outgoing_packets,
            incoming_ratio, outgoing_ratio, outgoing_std,
            outgoing_mean, packets_per_second, first_30_incoming,
            first_30_outgoing, inter_arrival_mean, inter_arrival_std,
            concentration_mean, concentration_std, alternative_sum
        ]
        features.append(feature_vector)

    return np.array(features)

# Monitored와 Unmonitored 데이터에서 피처 추출
features_mon = extract_features(X1_mon, X2_mon)
features_unmon = extract_features(X1_unmon, X2_unmon)

print(f"Monitored features shape: {features_mon.shape}")
print(f"Unmonitored features shape: {features_unmon.shape}")


Monitored features shape: (19000, 15)
Unmonitored features shape: (3000, 15)


In [11]:
import pandas as pd

# 피처 이름 정의 (15개 피처)
feature_names = [
    "Total Packets", "Incoming Packets", "Outgoing Packets", 
    "Incoming Ratio", "Outgoing Ratio", "Outgoing Std", 
    "Outgoing Mean", "Packets per Second", "First 30 Incoming", 
    "First 30 Outgoing", "Inter-arrival Mean", "Inter-arrival Std", 
    "Concentration Mean", "Concentration Std", "Alternative Sum"
]
# X 데이터를 DataFrame으로 변환
df = pd.DataFrame(features_mon, columns=feature_names)
df["Label"] = y_mon
print(df.head())  # 데이터 확인

# CSV 파일로 저장하기
df.to_csv("mon.csv", index=False, encoding="utf-8-sig")  # index=False는 행 번호를 제외하고 저장하기 위함입니다.


   Total Packets  Incoming Packets  Outgoing Packets  Incoming Ratio  \
0         1421.0             121.0            1300.0        0.085151   
1          518.0              80.0             438.0        0.154440   
2         1358.0             118.0            1240.0        0.086892   
3         1446.0             122.0            1324.0        0.084371   
4         1406.0             115.0            1291.0        0.081792   

   Outgoing Ratio  Outgoing Std  Outgoing Mean  Packets per Second  \
0        0.914849    515.483953     773.322314          140.138067   
1        0.845560    139.231951     226.162500           50.984252   
2        0.913108    472.735508     786.110169          122.232223   
3        0.915629    513.916038     820.139344          108.233533   
4        0.918208    503.993490     789.608696          132.142857   

   First 30 Incoming  First 30 Outgoing  Inter-arrival Mean  \
0                9.0               21.0            0.007141   
1                8.0

In [ ]:
import numpy as np

# Monitored와 Unmonitored 데이터 병합
X = np.vstack((features_mon, features_unmon))
y = np.hstack((y_mon, y_unmon))

print(f"Total data shape: {X.shape}, Total labels: {y.shape}")


In [ ]:
import pandas as pd

# 피처 이름 정의 (15개 피처)
feature_names = [
    "Total Packets", "Incoming Packets", "Outgoing Packets", 
    "Incoming Ratio", "Outgoing Ratio", "Outgoing Std", 
    "Outgoing Mean", "Packets per Second", "First 30 Incoming", 
    "First 30 Outgoing", "Inter-arrival Mean", "Inter-arrival Std", 
    "Concentration Mean", "Concentration Std", "Alternative Sum"
]

# X 데이터를 DataFrame으로 변환
df = pd.DataFrame(X, columns=feature_names)
df["Label"] = y
print(df.head())  # 데이터 확인


In [ ]:
# CSV 파일로 저장하기
df.to_csv("monunmon.csv", index=False, encoding="utf-8-sig")  # index=False는 행 번호를 제외하고 저장하기 위함입니다.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 히스토그램 생성 (x축, y축 레이블 추가 및 bin 조정)
fig, axes = plt.subplots(5, 4, figsize=(20, 20))  # 5x4 그리드 생성
fig.suptitle("Feature Distributions", fontsize=24)  # 메인 제목 설정

# 각 피처에 대해 히스토그램 생성
for i, feature in enumerate(df.columns[:-1]):  # 마지막 열(Label)은 제외
    ax = axes[i // 4, i % 4]  # 그리드 위치 계산
    
    # 히스토그램 생성
    sns.histplot(data=df, x=feature, bins=50, ax=ax, kde=False)  # bins=50으로 더 촘촘하게
    ax.set_title(feature, fontsize=14)  # 각 그래프의 제목 설정
    ax.set_xlabel(f"{feature} 값", fontsize=12)  # x축 레이블 설정
    ax.set_ylabel("빈도(Frequency)", fontsize=12)  # y축 레이블 설정

    # 특정 피처의 경우 x축 범위를 수동으로 설정 (예: 패킷 수가 큰 경우)
    if feature in ["Total Packets", "Incoming Packets", "Outgoing Packets"]:
        ax.set_xlim(0, 10000)  # 0~10,000 범위로 설정

# 레이아웃 조정 (제목과 그래프가 겹치지 않게)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
plt.figure(figsize=(100, 6))
sns.boxplot(data=df, x="Label", y="Total Packets")
plt.title("Total Packets by Label", fontsize=16)
plt.show()

In [ ]:
# 피처 간 상관관계 계산
corr_matrix = df.corr()

# 히트맵 생성
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Heatmap", fontsize=16)
plt.show()